In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle
import matplotlib
import ipyparams

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import func_optimize as fo
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
import neurolib.dashboard.layout as layout
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

cmap_ = layout.getcolormap()
darkgrey, midgrey, lightgrey, color_bi_updown, color_LC, color_bi_uposc = layout.getcolors_rgba()
exc_red = cmap_(3)
inh_blue=cmap_(0)
green = cmap_(2)

<IPython.core.display.Javascript object>

not here


In [2]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [3]:
points = np.array( [ [0.4, 0.322] ])

# 10 points, two costs, 4 methods, N 

In [4]:
def create_dict(task, point, cost_params, init, init_vars, target, sigma_array, N_max_array, max_it):
    return dict( {'task':task, 'point':point, 'cost_params':cost_params, 'init':init, 'init_vars':init_vars,
            'target':target, 'sigma':sigma_array, 'N_array':N_max_array,
            'cost':np.zeros(( len(sigma_array), 4, 2, len(N_max_array), max_it )) } )

params_list = ['task', 'point', 'cost_params', 'init', 'init_vars', 'target', 'sigma', 'N_array', 'cost']

In [6]:
dict_list = []
sigma_array = np.arange(0.01, 0.031, 0.01)
N_array = [1,2,10,30]
maxit = 30

for p in points:
    dict_list.append(create_dict('DU', p, [None]*len(sigma_array), [1], None,
                [None]*len(sigma_array), sigma_array, N_array, maxit) )

In [7]:
for d in dict_list:
    print(d['target'])

[None, None, None]


In [10]:
dict_file = "OU_1I.pickle"

if False:
    file=open(dict_file, "wb")
    pickle.dump(dict_list, file)
    file.close()

if True:
    file = open(dict_file, "rb")
    dict_list = pickle.load(file)
    file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'OU_1I.pickle'

In [8]:
dur_pre = 10
dur_post = 10
dur = 500.
dur_comp = 2000.
dt = aln.params.dt
T = int(1 + dur / dt)

prec_vars = [0,1]
trans_time = 0.8

n_pre = int(np.around(dur_pre/aln.params.dt,1))
n_post = int(np.around(dur_post/aln.params.dt,1))

tol = 1e-16
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

maxC = [5., -5., 0.18, 0.]
n_dur = int(np.around(dur/aln.params.dt + 1.,1))
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

noise_realizations_final = 10000
noise_realizations_target_compute = 1000
init_it = 5

In [11]:
# get init vars without noise
aln = ALNModel()
data.set_parameters(aln)
aln.params.duration = dur_comp

# initial without noise
control0 = aln.getZeroControl()

for d in dict_list:
    
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    aln.params.duration = dur_comp
    
    if type(d['init_vars']) == type(None):
    
        control0[0,0,:500] = -5.

        aln.run(control=control0)

        init_state_vars = np.zeros(( len(state_vars) ))
        for j in range(len(state_vars)):
            if aln.state[state_vars[j]].size == 1:
                init_state_vars[j] = aln.state[state_vars[j]][0]
            else:
                init_state_vars[j] = aln.state[state_vars[j]][0,-1]

        initVars = init_state_vars.copy()
        d['init_vars'] = initVars
    
    for ind_s in range(len(d['sigma'])):
        
        #if type(d['target'][ind_s]) != type(None):
        #    continue        
        
        s = d['sigma'][ind_s]
        aln.params.sigma_ou = s
        aln.params.duration = dur_comp
        control0[0,0,:500] = 5.
        
        stateswitchcount = 0
        target_rates_e = []
        target_rates_i = []

        for j in range(noise_realizations_target_compute):
            if j % 100 == 0:
                print(j)
            aln.run(control=control0)
            #plt.plot(aln.t, aln.rates_exc[0,:])
            if np.amin(aln.rates_exc[0,1000:]) < 3.:
                #print("fell in down state, repeat")
                j -= 1
                stateswitchcount += 1
                if stateswitchcount > 0.8 * noise_realizations_target_compute:
                    print("UNSTABLE: point, sigma ", d['point'], s)
                    break
                continue
            target_rates_e.append(aln.rates_exc[0,1000:])
            target_rates_i.append(aln.rates_inh[0,1000:])
        
        #plt.show()            

        aln.params.duration = dur
        target = aln.getZeroTarget()
        target[:,0,:] = np.mean(target_rates_e)
        target[:,1,:] = np.mean(target_rates_i)
        stddev = np.std(target_rates_e) + np.std(target_rates_i)
        
        d['target'][ind_s] = target
        d['cost_params'][ind_s] = [1. / ( stddev * (1. - trans_time) * dur ), 0., 1.]
        
        #print(stddev)
        
    file=open(dict_file, "wb")
    pickle.dump(dict_list, file)
    file.close()

Point =  [0.4   0.322]
0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900


In [12]:
dict_list[0]['init_vars']

array([ 1.38486146e+000,  1.28893451e+001,  2.55707316e-001,
        6.95303580e-001,  0.00000000e+000,  1.71554859e-001,
        6.33669846e-001,  1.62019343e-001,  7.80253539e-001,
        7.77047142e-003,  7.06637087e-003,  7.00922914e-003,
        6.73992948e-003,  1.32914953e-176,  9.96862146e-177,
        1.62431654e+000,  1.55922906e+000, -6.04480567e+001,
        1.37574716e+001,  4.77734640e+000])

In [17]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 1
method_ind = 0
step_method_ind = 0

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        aln.params.duration = dur
        zero_init = aln.getZeroControl()
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
        
        if np.amin(d['cost'][s_ind, method_ind, step_method_ind, :, 0]) == 0.:
        
            setinit(d['init_vars'], aln)
            cost_final = 0.

            for n_ in range(noise_realizations_final):
                state_ = fo.updateState(aln, zero_init)
                cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                               state_, target_, list(prec_vars) )

            d['cost'][s_ind, :, :, :, 0] = cost_final/noise_realizations_final
            
        print('cost = ', d['cost'][s_ind, 0, 0, 0])    
        
        for i_it in range(1, d['cost'].shape[4], 1):
            
            print("iteration = ", i_it)
        
            for n_ind in range(len(d['N_array'])):

                if d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.:
                    continue
                    
                n_max = d['N_array'][n_ind]
                print("n max = ", n_max)

                aln.params.duration = dur
                control_init = aln.getZeroControl()

                if i_it <= init_it:
                    c_vars = d['init']
                else:
                    c_vars = [0,1]
                    
                n_ = 0
                control_sum = aln.getZeroControl()
                
                while n_ < n_max:
                
                    print('--------', n_)

                    aln.params.duration = dur
                    setinit(d['init_vars'], aln)

                    bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                        control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = i_it,
                        tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                        t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                        control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                        noise_real=0, init_params = d['init_vars'])
                    
                    control_sum += bc[:,:,n_pre:-n_post]
                    n_ += 1

                control_init = control_sum / n_max
        
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close() 

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008312694165372206 0.0 1.0
cost =  [1387.73653004   26.8638452    14.49127347   50.64301482   65.84630241
   12.94640511  460.80147254  248.22270297   80.39297747   11.89300995
   60.8430125    70.73490131  125.26229946  276.54943782   19.71277114
   11.94734178   49.31718727  548.60741207   67.33283654    3.40453019
  122.04849434  228.70728209   21.1896813     1.9013883   203.16581711
  844.76281837  249.09531393    2.66328909   20.71035066   45.64873337]
iteration =  1
iteration =  2
iteration =  3
iteration =  4
iteration =  5
iteration =  6
iteration =  7
iteration =  8
iteration =  9
iteration =  10
iteration =  11
iteration =  12
iteration =  13
iteration =  14
iteration =  15
iteration =  16
iteration =  17
iteration =  18
iteration =  19
iteration =  20
iteration =  21
iteration =  22
iteration =  23
iteration =  24
iteration =  25
iteration =  26
iteration =  27
iteration =  28
iteration =  29
sigma =  0.02
set cost par

In [29]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 2
method_ind = 1
step_method_ind = 0

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
            control_init = aln.getZeroControl()
        
            for i_it in range(1, d['cost'].shape[4], 1):
                
                if d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.:
                    continue
                
                print("iteration = ", i_it)

                if i_it <= init_it:
                    c_vars = d['init']
                else:
                    c_vars = [0,1]
                    
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = i_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M2')
            
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, bc[:,:,100:-100])

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, bc[:,:,100:-100])
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008312694165372206 0.0 1.0
n =  1
n =  2
n =  10
n =  30
sigma =  0.02
set cost params:  0.0041518937810734345 0.0 1.0
n =  1
n =  2
n =  10
n =  30
iteration =  9
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  667.5791945606836
Gradient descend method:  None
RUN  1 , total integrated cost =  5.059276772722926
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size to

step size too small or too large
step size too small or too large
RUN  5 , total integrated cost =  6.217673884007994
RUN  6 , total integrated cost =  6.3043218914676995
no cost improvement
RUN  7 , total integrated cost =  3.8208218931073454
RUN  7 , total integrated cost mean =  3.8208218931073454
Improved over  7  iterations in  273.16045102701173  seconds by  99.37159626515334  percent.
cost =  3.439771340126617
iteration =  11
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  662.208732621602
Gradient descend method:  None
RUN  1 , total integrated cost =  13.782705475317522
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too 

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  3 , total integrated cost =  3.5509359193806347
step size too small or too large
RUN  4 , total integrated cost =  3.426404177194867
RUN  5 , total integrated cost =  3.8602081361109937
no cost improvement
RUN  6 , total integrated cost =  2.8620416032681715
RUN  6 , total integrated cost mean =  2.8620416032681715
Improved over  6  iterations in  155.7249722990091  seconds by  99.56879603878966  percent.
cost =  2.8708238382007343
iteration =  14
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  659.9208822498805
Gradient descend method:  None
RUN  1 , total integrated cost =  31.223103349240855
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step s

RUN  12 , total integrated cost =  1.9477931960054842
RUN  13 , total integrated cost =  1.852023227643459
RUN  14 , total integrated cost =  1.8593126041684052
step size too small or too large
step size too small or too large
RUN  15 , total integrated cost =  1.8411545047626932
step size too small or too large
step size too small or too large
step size too small or too large
RUN  16 , total integrated cost =  1.7472527086774456
RUN  16 , total integrated cost mean =  1.7472527086774456
Improved over  16  iterations in  204.27957663399866  seconds by  99.73758976599974  percent.
cost =  2.677278559400275
iteration =  17
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  617.6386448271895
Gradient descend method:  None
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size 

...

cost =  18.778598297585564
iteration =  12
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  351.03015350411675
Gradient descend method:  None
RUN  1 , total integrated cost =  18.135401514272377
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  2 , total integrated cost =  1.8580406593758199
RUN  3 , total integrated cost =  1.

RUN  2 , total integrated cost =  2.288290074466423
step size too small or too large
RUN  3 , total integrated cost =  2.1326506411055575
step size too small or too large
step size too small or too large
step size too small or too large
RUN  4 , total integrated cost =  1.9990336320037974
RUN  5 , total integrated cost =  2.0572010265099703
RUN  6 , total integrated cost =  1.9118001474318371
step size too small or too large
RUN  7 , total integrated cost =  1.9838335017595554
RUN  8 , total integrated cost =  1.7789539914226302
step size too small or too large
RUN  9 , total integrated cost =  1.7888970384892007
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  10 , total integrated cost =  1.6959412404000405
RUN  11 , total integrated cost =  1.7692893726767467
RUN  12 , total integrated cost =  1.8096831857937885
RUN  13 , total integrated cost =  1.996887999689383

step size too small or too large
RUN  10 , total integrated cost =  3.6046763078707103
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  11 , total integrated cost =  3.589465095000534
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  12 , total integrated cost =  3.4324831383094905
step size too small or too large
step size too small or too large
RUN  13 , total integrated cost =  3.4151711470496626
step size too small or too large
RUN  14 , total integrated cost =  3.377557811958925
RUN  15 , total integrated cost =  3.2767180213826657
RUN  16 , total integrated cost =  3.248299515308839
RUN  17 , total integrated cost =  3.2148031176971794
RUN  18 , total integrated cost =  3.16388678168846
step size too small or too large
step size too small or t

step size too small or too large
step size too small or too large
RUN  18 , total integrated cost =  1.967759372704606
RUN  19 , total integrated cost =  1.9877094508593167
step size too small or too large
RUN  20 , total integrated cost =  1.9151071455155242
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  24 , total integrated cost mean =  1.779040004297379
Improved over  24  iterations in  283.40157385598286  seconds by  99.38776469281622  percent.
cost =  10.685201239216084
iteration =  25
interpolate adjoi

cost =  6.049659154776618
iteration =  29
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  285.0207530473832
Gradient descend method:  None
step size too small or too large
RUN  1 , total integrated cost =  4.446401678669134
RUN  2 , total integrated cost =  1.8531258654693479
RUN  3 , total integrated cost =  2.111450837241809
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  4 , total integrated cost =  2.324413062319423
RUN  5 , total integrated cost =  2.2279781730707096
step size too small or too large
RUN  6 , total integrated cost =  2.4899065011167774
RUN  7 , total integrated cost =  2.7132390989606185
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  8 , total integrated cost =  2.3652

In [31]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 3
method_ind = 2
step_method_ind = 0

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
            control_init = aln.getZeroControl()
        
            for i_it in range(1, d['cost'].shape[4], 1):
                
                if d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.:
                    continue
                
                print("iteration = ", i_it)

                if i_it <= init_it:
                    c_vars = d['init']
                else:
                    c_vars = [0,1]
                    
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = i_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3')
            
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, bc[:,:,100:-100])

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, bc[:,:,100:-100])
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close() 

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008312694165372206 0.0 1.0
n =  1
n =  2
n =  10
n =  30
iteration =  20
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1388.8910226322528
Gradient descend method:  None
RUN  1 , total integrated cost =  128.30392851628218
step size too small or too large
step size too small or too large
RUN  2 , total integrated cost =  8.22565665419344
RUN  3 , total integrated cost =  1.557432410143206
step size too small or too large
RUN  4 , total integrated cost =  1.4690845953676106
step size too small or too large
step size too small or too large
RUN  5 , total integrated cost =  1.2981627516586856
step size too small or too large
step size too small or too large
RUN  6 , total integrated cost =  1.0880366540488287
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step

KeyboardInterrupt: 

In [ ]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 4
method_ind = 3
step_method_ind = 0

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
            control_init = aln.getZeroControl()
        
            for i_it in range(1, d['cost'].shape[4], 1):
                
                if d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.:
                    continue
                
                print("iteration = ", i_it)

                if i_it <= init_it:
                    c_vars = d['init']
                else:
                    c_vars = [0,1]
                    
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = i_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3')
            
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, bc[:,:,100:-100])

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, bc[:,:,100:-100])
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close() 

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008312694165372206 0.0 1.0
n =  1
iteration =  1
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1425.6943228376078
Gradient descend method:  None
RUN  1 , total integrated cost =  6.435674825525409
RUN  1 , total integrated cost mean =  6.435674825525409
Improved over  1  iterations in  0.2206974650034681  seconds by  99.54859364153766  percent.
cost =  134.18404485317646
iteration =  2
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1340.853604171989
Gradient descend method:  None
RUN  1 , total integrated cost =  3.6693198289484985
RUN  2 , total integrated cost =  2.424853640836297
RUN  2 , total integrated cost mean =  2.424853640836297
Improved over  2  iterations in  0.3032175329863094  seconds by  99.81915597397871  percent.
cost =  2.5203435807077277
iteration =  3
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1411.8497097060513
Gradient descend me

RUN  2 , total integrated cost =  2.105539253535593
RUN  3 , total integrated cost =  2.2261296621436233
RUN  4 , total integrated cost =  2.0745440954755727
RUN  5 , total integrated cost =  2.2819104423673116
RUN  6 , total integrated cost =  1.7870600977485522
RUN  7 , total integrated cost =  1.7155519687360314
RUN  8 , total integrated cost =  1.569928041539109
RUN  9 , total integrated cost =  1.7008106021297997
RUN  10 , total integrated cost =  1.567911098504683
RUN  11 , total integrated cost =  1.6620142436499261
RUN  12 , total integrated cost =  1.5085641487537078
RUN  13 , total integrated cost =  1.6017414652720534
RUN  13 , total integrated cost mean =  1.6017414652720534
Improved over  13  iterations in  3.4410867340047844  seconds by  99.88511480097317  percent.
cost =  4.8422621955388045
iteration =  14
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1376.0612697154193
Gradient descend method:  None
RUN  1 , total integrated cost =  1.969341089

RUN  9 , total integrated cost =  1.2965467586951598
RUN  10 , total integrated cost =  1.423880796942723
RUN  11 , total integrated cost =  1.5223491023659932
RUN  12 , total integrated cost =  1.4029968290690413
RUN  13 , total integrated cost =  2.0282314078132586
RUN  14 , total integrated cost =  1.6926399358606927
no cost improvement
RUN  15 , total integrated cost =  1.9860881920896238
RUN  16 , total integrated cost =  1.958730634484815
RUN  17 , total integrated cost =  1.851154392056012
RUN  18 , total integrated cost =  2.004309045980993
RUN  19 , total integrated cost =  1.8607919411873701
RUN  20 , total integrated cost =  1.6738855006130755
RUN  20 , total integrated cost mean =  1.6738855006130755
Improved over  20  iterations in  10.082721685001161  seconds by  99.87886428380965  percent.
cost =  3.081272621306068
iteration =  21
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1377.0280709056383
Gradient descend method:  None
RUN  1 , total integ

RUN  1 , total integrated cost =  2.090839893355062
RUN  2 , total integrated cost =  1.9266030113469106
RUN  3 , total integrated cost =  1.948328115327064
RUN  4 , total integrated cost =  1.6801034499990066
RUN  5 , total integrated cost =  1.3976629042382804
RUN  6 , total integrated cost =  1.5239822690617997
RUN  7 , total integrated cost =  1.407662844202575
RUN  8 , total integrated cost =  1.4802711367446757
RUN  9 , total integrated cost =  1.6033631190768183
RUN  10 , total integrated cost =  1.34281827000191
RUN  11 , total integrated cost =  1.4222131633616897
RUN  12 , total integrated cost =  1.3099302518153866
RUN  13 , total integrated cost =  1.3867978201286522
RUN  14 , total integrated cost =  1.2485333318635221
RUN  15 , total integrated cost =  1.3804588568648626
RUN  16 , total integrated cost =  1.1975859682344556
RUN  17 , total integrated cost =  1.216009687907623
RUN  18 , total integrated cost =  1.3204040939058306
RUN  19 , total integrated cost =  1.266284

RUN  1 , total integrated cost =  30.688270603881985
RUN  2 , total integrated cost =  1.8288302060687118
RUN  3 , total integrated cost =  1.7794878281384552
RUN  4 , total integrated cost =  1.8293145933061445
RUN  5 , total integrated cost =  1.6406043137419242
RUN  6 , total integrated cost =  1.5682326422962871
RUN  7 , total integrated cost =  2.491909064536964
RUN  8 , total integrated cost =  2.0478906700618906
RUN  9 , total integrated cost =  2.195008583051127
RUN  10 , total integrated cost =  2.016280415802027
RUN  10 , total integrated cost mean =  2.016280415802027
Improved over  10  iterations in  8.36566660698736  seconds by  99.85585808999082  percent.
cost =  2.1615740367843204
iteration =  11
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1355.5384216853463
Gradient descend method:  None
RUN  1 , total integrated cost =  43.871606287256206
RUN  2 , total integrated cost =  2.488607036512099
RUN  3 , total integrated cost =  2.6533490002979465

...

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size 

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  5 , total integrated cost =  9.10271134380406
step size too small or too large
RUN  6 , total integrated cost =  3.566724461814648
RUN  7 , total integrated cost =  3.250179818266526
RUN  8 , total integrated cost =  2.9100522184650406
RUN  9 , total integrated cost =  2.5732745268455774
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  10 , tot

step size too small or too large
RUN  9 , total integrated cost =  2.2607108575076613
RUN  10 , total integrated cost =  2.2509242167321974
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  11 , total integrated cost =  2.3436136238751235
RUN  12 , total integrated cost =  2.3831789389926397
RUN  13 , total integrated cost =  2.33195790808004
RUN  14 , total integrated cost =  1.9261054936004884
RUN  15 , total integrated cost =  16.538265627309244
no cost improvement
RUN  16 , total integrated cost =  2.995592905212736
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or t

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  9 , total integrated cost =  18.265861514110487
step size too small or too large
RUN  10 , total integrated cost =  2.1781937556408035
RUN  11 , total integrated cost =  2.1193394257779636
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too 

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  13 , total integrated cost =  2.255911859305959
step size too small or too large
RUN  14 , total integrated cost =  2.2740718201592647
step size too small or too large
step size too small or too large
step size too small or too large
st

In [22]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 1
method_ind = 0
step_method_ind = 1
step_m = 'S2'

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        aln.params.duration = dur
        zero_init = aln.getZeroControl()
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
        
        if np.amin(d['cost'][s_ind, method_ind, step_method_ind, :, 0]) == 0.:
        
            setinit(d['init_vars'], aln)
            cost_final = 0.

            for n_ in range(noise_realizations_final):
                state_ = fo.updateState(aln, zero_init)
                cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                               state_, target_, list(prec_vars) )

            d['cost'][s_ind, :, :, :, 0] = cost_final/noise_realizations_final
            
        print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, :, :])    
        
        for i_it in range(1, d['cost'].shape[4], 1):
            
            print("iteration = ", i_it)
        
            for n_ind in range(len(d['N_array'])):

                if d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.:
                    continue
                    
                n_max = d['N_array'][n_ind]
                print("n max = ", n_max)

                aln.params.duration = dur
                control_init = aln.getZeroControl()

                if i_it <= init_it:
                    c_vars = d['init']
                else:
                    c_vars = [0,1]
                    
                n_ = 0
                control_sum = aln.getZeroControl()
                
                while n_ < n_max:
                
                    print('--------', n_)

                    aln.params.duration = dur
                    setinit(d['init_vars'], aln)

                    bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                        control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = i_it,
                        tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                        t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                        control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                        noise_real=0, init_params = d['init_vars'], method_step=step_m)
                    
                    control_sum += bc[:,:,n_pre:-n_post]
                    n_ += 1

                control_init = control_sum / n_max
        
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close() 

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008312694165372206 0.0 1.0
cost =  [[1387.73653004    0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.        ]
 [1387.73653004    0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.        ]
 [1387.73653004    0.            0.            0.            0.
     0.            0.            0.            0.         

RUN  1 , total integrated cost =  9.919303390094326
RUN  1 , total integrated cost =  9.919303390094326
Improved over  1  iterations in  0.11532189798890613  seconds by  99.28929974966555  percent.
-------- 6
interpolate adjoint :  True True True
initial cost =  1362.1992351096028
RUN  0 , total integrated cost =  1362.1992351096028
Gradient descend method:  None
RUN  1 , total integrated cost =  1.969950656791072
RUN  1 , total integrated cost =  1.969950656791072
Improved over  1  iterations in  0.1210180810012389  seconds by  99.8553845424357  percent.
-------- 7
interpolate adjoint :  True True True
initial cost =  1394.2209051121374
RUN  0 , total integrated cost =  1394.2209051121374
Gradient descend method:  None
RUN  1 , total integrated cost =  3.5888079721279156
RUN  1 , total integrated cost =  3.5888079721279156
Improved over  1  iterations in  0.11458379100076854  seconds by  99.74259402086362  percent.
-------- 8
interpolate adjoint :  True True True
initial cost =  1293.

RUN  1 , total integrated cost =  3.894259000961828
RUN  1 , total integrated cost =  3.894259000961828
Improved over  1  iterations in  0.12673603201983497  seconds by  99.71743337437373  percent.
cost =  496.6269942385499
iteration =  2
n max =  1
-------- 0
interpolate adjoint :  True True True
initial cost =  1404.2933841485888
RUN  0 , total integrated cost =  1404.2933841485888
Gradient descend method:  None
RUN  1 , total integrated cost =  6.74873104485774
RUN  2 , total integrated cost =  5.94564346265088
RUN  2 , total integrated cost =  5.94564346265088
Improved over  2  iterations in  0.2454966879740823  seconds by  99.57660959385238  percent.
cost =  8.461311847774686
n max =  2
-------- 0
interpolate adjoint :  True True True
initial cost =  1296.4114521778229
RUN  0 , total integrated cost =  1296.4114521778229
Gradient descend method:  None
RUN  1 , total integrated cost =  10.046187261476634
RUN  2 , total integrated cost =  1.6309475212345115
RUN  2 , total integrated

RUN  2 , total integrated cost =  1.8035619919186994
RUN  2 , total integrated cost =  1.8035619919186994
Improved over  2  iterations in  0.226836401008768  seconds by  99.87391577433651  percent.
-------- 7
interpolate adjoint :  True True True
initial cost =  1388.5471226367397
RUN  0 , total integrated cost =  1388.5471226367397
Gradient descend method:  None
RUN  1 , total integrated cost =  15.62310730054232
RUN  2 , total integrated cost =  2.1382586542490483
RUN  2 , total integrated cost =  2.1382586542490483
Improved over  2  iterations in  0.24530274301650934  seconds by  99.84600748369355  percent.
-------- 8
interpolate adjoint :  True True True
initial cost =  1404.6445390200727
RUN  0 , total integrated cost =  1404.6445390200727
Gradient descend method:  None
RUN  1 , total integrated cost =  16.38078004803362
RUN  2 , total integrated cost =  1.7864824431152615
RUN  2 , total integrated cost =  1.7864824431152615
Improved over  2  iterations in  0.22692068500327878  se

RUN  2 , total integrated cost =  3.3459946059471766
RUN  2 , total integrated cost =  3.3459946059471766
Improved over  2  iterations in  0.2485043260094244  seconds by  99.74478880659716  percent.
-------- 27
interpolate adjoint :  True True True
initial cost =  1418.2940088061814
RUN  0 , total integrated cost =  1418.2940088061814
Gradient descend method:  None
RUN  1 , total integrated cost =  1.5484368373032322
RUN  2 , total integrated cost =  5.565371113914519
RUN  2 , total integrated cost =  5.565371113914519
Improved over  2  iterations in  0.2462748110119719  seconds by  99.60760102775876  percent.
-------- 28
interpolate adjoint :  True True True
initial cost =  1400.6219601367911
RUN  0 , total integrated cost =  1400.6219601367911
Gradient descend method:  None
RUN  1 , total integrated cost =  6.962090772734669
RUN  2 , total integrated cost =  1.8394392917676041
RUN  2 , total integrated cost =  1.8394392917676041
Improved over  2  iterations in  0.2525550670106895  se

...

RUN  4 , total integrated cost =  1.5865577863722573
RUN  5 , total integrated cost =  1.8845295141503562
RUN  6 , total integrated cost =  1.6862152123241971
RUN  7 , total integrated cost =  1.5966093983560077
RUN  8 , total integrated cost =  1.520391258595222
RUN  9 , total integrated cost =  1.222335777452299
RUN  10 , total integrated cost =  1.3804295770378787
RUN  11 , total integrated cost =  1.1918923216109163
RUN  12 , total integrated cost =  1.1842688443794074
RUN  13 , total integrated cost =  1.2331893676291195
RUN  14 , total integrated cost =  1.1617315513204274
RUN  14 , total integrated cost =  1.1617315513204274
Improved over  14  iterations in  4.602333171991631  seconds by  99.81555087961421  percent.
-------- 11
interpolate adjoint :  True True True
initial cost =  661.4550063367994
RUN  0 , total integrated cost =  661.4550063367994
Gradient descend method:  None
RUN  1 , total integrated cost =  4.749258224635285
RUN  2 , total integrated cost =  4.393221749444

RUN  3 , total integrated cost =  2.1809918832557966
RUN  4 , total integrated cost =  1.6282234843526047
RUN  5 , total integrated cost =  1.5551392517855651
RUN  6 , total integrated cost =  1.6404888882408928
RUN  7 , total integrated cost =  1.380039863801782
RUN  8 , total integrated cost =  1.1634132210831676
RUN  9 , total integrated cost =  1.129380437109439
RUN  10 , total integrated cost =  1.2199717333755158
RUN  11 , total integrated cost =  1.171407983229268
RUN  12 , total integrated cost =  1.3262184040786842
RUN  13 , total integrated cost =  1.1596034026687967
RUN  14 , total integrated cost =  1.0983604719076498
RUN  14 , total integrated cost =  1.0983604719076498
Improved over  14  iterations in  2.9568468059878796  seconds by  99.84294248382649  percent.
-------- 19
interpolate adjoint :  True True True
initial cost =  712.8078242696319
RUN  0 , total integrated cost =  712.8078242696319
Gradient descend method:  None
RUN  1 , total integrated cost =  2.75581780658

RUN  1 , total integrated cost =  1.3624223800572777
RUN  2 , total integrated cost =  1.6496276726575805
RUN  3 , total integrated cost =  1.578241741110193
RUN  4 , total integrated cost =  1.5840130098355854
RUN  5 , total integrated cost =  1.474000468057155
RUN  6 , total integrated cost =  1.2364315983177308
RUN  7 , total integrated cost =  1.335070700434502
RUN  8 , total integrated cost =  1.1546478463359688
RUN  9 , total integrated cost =  1.2830645241372327
RUN  10 , total integrated cost =  1.3749379262472312
RUN  11 , total integrated cost =  1.3768762721957324
RUN  12 , total integrated cost =  1.3792817859697462
RUN  13 , total integrated cost =  1.2804002457082866
RUN  14 , total integrated cost =  1.4001775874010427
RUN  14 , total integrated cost =  1.4001775874010427
Improved over  14  iterations in  3.1711424039967824  seconds by  99.78733208954903  percent.
-------- 27
interpolate adjoint :  True True True
initial cost =  693.0032823789855
RUN  0 , total integrate

RUN  13 , total integrated cost =  4.61469337748487
RUN  14 , total integrated cost =  2.1405236852998977
RUN  15 , total integrated cost =  2.0695794265212566
RUN  15 , total integrated cost =  2.0695794265212566
Improved over  15  iterations in  4.508876536012394  seconds by  99.68278123415351  percent.
-------- 1
interpolate adjoint :  True True True
initial cost =  682.9742767034795
RUN  0 , total integrated cost =  682.9742767034795
Gradient descend method:  None
RUN  1 , total integrated cost =  8.4618476304926
RUN  2 , total integrated cost =  3.8105540080208993
RUN  3 , total integrated cost =  2.9170526113470157
RUN  4 , total integrated cost =  2.148329048448871
RUN  5 , total integrated cost =  2.2326083117776085
RUN  6 , total integrated cost =  2.2228676211514777


KeyboardInterrupt: 

In [ ]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 2
method_ind = 1
step_method_ind = 1

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
            control_init = aln.getZeroControl()
        
            for i_it in range(1, d['cost'].shape[4], 1):
                
                if d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.:
                    continue
                
                print("iteration = ", i_it)

                if i_it <= init_it:
                    c_vars = d['init']
                else:
                    c_vars = [0,1]
                    
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = i_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M2', method_step=step_m)
            
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, bc[:,:,100:-100])

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, bc[:,:,100:-100])
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()